The following additional libraries are needed to run this
notebook. Note that running on Colab is experimental, please report a Github
issue if you have any problem.

In [ ]:
!pip install -U mxnet-cu101==1.7.0


# Diferenciação automática
:label:`sec_autograd`

Como já explicado em :numref:`sec_calculus`, a diferenciação é uma etapa crucial em quase todos os algoritmos de otimização de *Deep Learning*. Embora os cálculos para obter esses derivados sejam diretos, exigindo apenas alguns cálculos básicos, para modelos complexos, trabalhando as atualizações manualmente pode ser uma tarefa difícil (e muitas vezes sujeita a erros).
*Frameworks* de *Deep learning* aceleram este trabalho calculando automaticamente as derivadas, ou seja, *diferenciação automática*. Na prática, com base em nosso modelo projetado o sistema constrói um *grafo computacional*, rastreando quais dados combinados por meio de quais operações produzem a saída. A diferenciação automática permite que o sistema propague gradientes posteriormente. Aqui, propagar(do Inglês *backpropagate*) significa simplesmente traçar o gráfico computacional, preencher as derivadas parciais em relação a cada parâmetro.


## Um exemplo simples

Como exemplo, digamos que estamos interessados em (**derivar a função $y = 2\mathbf{x}^{\top}\mathbf{x}$ com respeito ao vetor coluna $\mathbf{x}$.**)
Inicialmente criamos a variável `x` e atribuimos a ela um valor inicial.


In [1]:
from mxnet import autograd, np, npx

npx.set_np()

x = np.arange(4.0)
x

array([0., 1., 2., 3.])

[**Antes de calcularmos o gradiente de$y$ em relação a $\mathbf{x}$,
precisamos armazena-lo.**]
É importante que não aloquemos nova memória cada vez que tomamos uma derivada em relação a um parâmetro porque costumamos atualizar os mesmos parâmetros milhares ou milhões de vezes e podemos rapidamente ficar sem memória.
Observe que um gradiente de uma função com valor escalar com respeito a um vetor $\mathbf{x}$ tem valor vetorial e tem a mesma forma de $\mathbf{x}$.


In [2]:
# Alocamos memória do gradiente do vetor invocando `attach_grad`
x.attach_grad()
# Após calcularmos o gradiente como respeito a `x`, será possível
# acessa-lo vai atributo `grad`, cujo valor inicializará com 0s
x.grad

array([0., 0., 0., 0.])

(**Então calcularemos $y$.**)


In [3]:
# Colocaremos o código dentro do escopo `autograd.record` para contruir
# o grafo computacional
with autograd.record():
    y = 2 * np.dot(x, x)
y

array(28.)

Uma vez que `x` é um vetor de comprimento 4,
um produto interno de `x` e` x` é realizado,
produzindo a saída escalar que atribuímos a `y`.
Em seguida, [** podemos calcular automaticamente o gradiente de `y`
com relação a cada componente de `x` **]
chamando a função de retropropagação e imprimindo o gradiente.


In [4]:
y.backward()
x.grad

[03:58:07] src/base.cc:49: GPU context requested, but no GPUs found.


array([ 0.,  4.,  8., 12.])

(**O gradiente da função $y = 2\mathbf{x}^{\top}\mathbf{x}$
em relação a $\mathbf{x}$ should be $4\mathbf{x}$.**)
Vamos verificar rapidamente se nosso gradiente desejado foi calculado corretamente.


In [5]:
x.grad == 4 * x

array([ True,  True,  True,  True])

[**Agora calculamos outra função de `x`.**]


In [6]:
with autograd.record():
    y = x.sum()
y.backward()
x.grad  # Sobrescrito pelo novo gradiente calculado

array([1., 1., 1., 1.])

## Retroceder para variáveis não escalares

Tecnicamente, quando `y` não é um escalar,
a interpretação mais natural da diferenciação de um vetor `y`
em relação a um vetor, `x` é uma matriz.
Para `y` e` x` de ordem superior e dimensão superior,
o resultado da diferenciação pode ser um tensor de ordem alta.

No entanto, embora esses objetos mais exóticos apareçam
em aprendizado de máquina avançado (incluindo [**em *Deep Learning***]),
com mais frequência (**quando estamos retrocedendo um vetor,**)
estamos tentando calcular as derivadas das funções de perda
para cada constituinte de um *lote* de exemplos de treinamento.
Aqui, (**nossa intenção é**) não calcular a matriz de diferenciação
mas sim (**a soma das derivadas parciais
calculado individualmente para cada exemplo**) no lote.


In [7]:
# Quando invocamos `backward` em uma variável de vetor valorado `y` (em função de `x`),
# uma nova variável escalar é criada somando os elementos em `y`. Então o
# gradiente daquela variável escalar em respeito a `x` é computada
with autograd.record():
    y = x * x  # `y` is a vector
y.backward()
x.grad  # Igual a y = sum(x * x)

array([0., 2., 4., 6.])

## Computação *Detaching* 

Às vezes, desejamos [**mover alguns cálculos
fora do gráfico computacional registrado.**]
Por exemplo, digamos que `y` foi calculado como uma função de` x`,
e que subsequentemente `z` foi calculado como uma função de` y` e `x`.
Agora, imagine que quiséssemos calcular
o gradiente de `z` em relação a` x`,
mas queria, por algum motivo, tratar `y` como uma constante,
e apenas leve em consideração a função
que `x` jogou após` y` foi calculado.
Aqui, podemos desanexar `y` para retornar uma nova variável `u`
que tem o mesmo valor que `y`, mas descarta qualquer informação
sobre como `y` foi calculado no grafo computacional.
Em outras palavras, o gradiente não fluirá de volta de `u` para `x`.
Assim, a seguinte função de retropropagação calcula
a derivada parcial de `z = u * x` com respeito a` x` enquanto trata `u` como uma constante,
em vez da derivada parcial de `z = x * x * x` em relação a` x`.


In [8]:
with autograd.record():
    y = x * x
    u = y.detach()
    z = u * x
z.backward()
x.grad == u

array([ True,  True,  True,  True])

Uma vez que o cálculo de `y` foi registrado,
podemos subsequentemente invocar a retropropagação em `y` para obter a derivada de` y = x * x` com respeito a `x`, que é` 2 * x`.


In [9]:
y.backward()
x.grad == 2 * x

array([ True,  True,  True,  True])

## Computando o Gradiente do *Python Control Flow*

Uma vantagem de usar a diferenciação automática
é que [**mesmo se**] construir o gráfo computacional de (**uma função
requer muito trabalho com o uso do  *Python Control Flow***)
(por exemplo, condicionais, loops e chamadas de função arbitrárias),
(**ainda podemos calcular o gradiente da variável resultante.**)
No trecho a seguir, observe que
o número de iterações do loop `while`
e a avaliação da instrução `if`
ambos dependem do valor da entrada `a`.


In [10]:
def f(a):
    b = a * 2
    while np.linalg.norm(b) < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

Vamos computar o gradiente:


In [11]:
a = np.random.normal()
a.attach_grad()
with autograd.record():
    d = f(a)
d.backward()

Agora podemos analisar a função `f` definida acima.
Observe que é linear por partes em sua entrada `a`.
Em outras palavras, para qualquer `a` existe algum escalar constante` k`
tal que `f (a) = k * a`, onde o valor de` k` depende da entrada `a`.
Consequentemente, `d / a` nos permite verificar se o gradiente está correto.


In [12]:
a.grad == d / a

array(True)

## Sumário

* *Frameworks* de *Deep learning* podem automatizar o cálculo de derivadas. Para usá-lo, primeiro anexamos gradientes às variáveis em relação às quais desejamos as derivadas parciais. Em seguida, registramos o cálculo de nosso valor alvo, executamos sua função para retropropagação e acessamos o gradiente resultante.


## Exercícios

1. Por que a segunda derivada é muito mais computacionalmente cara de se calcular do que a primeira derivada?
2. Depois de executar a função de retropropagação, execute-a imediatamente novamente e veja o que acontece.
3. No exemplo de fluxo de controle onde calculamos a derivada de `d` com respeito a `a`, o que aconteceria se mudássemos a variável `a` para um vetor ou matriz aleatória. Neste ponto, o resultado do cálculo `f (a)` não é mais um escalar. O que acontece com o resultado? Como analisamos isso?
4. Redesenhe um exemplo para encontrar o gradiente do *Control Flow*. Execute e analise o resultado.
5. Seja $f (x) = \ sin (x)$. Plote $f (x)$ e $\ frac {df (x)} {dx}$, onde o último é calculado sem explorar que $f '(x) = \ cos (x)$.


[Discussions](https://discuss.d2l.ai/t/34)


<!--stackedit_data:
eyJoaXN0b3J5IjpbLTI4NzgwMTM5OSw4ODQwMzIxNTksLTg2Mj
IyMDIxOSwtMTg3MDI0NTA0NiwtMjAyMDM0OTY2NSwxMjQ4MDc0
NTIyLDE0MDY0MzkzNDFdfQ==
-->
